In [33]:
import feedparser
import json

import pandas as pd
import numpy as np

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

from tqdm import tqdm

In [34]:
# TODO create input for search f1 team or driver
rss_url = "https://news.google.com/rss/search?q=oscar%20piastri"

feed = feedparser.parse(rss_url)  # this returns a dictionary

In [35]:
items = feed.entries

In [41]:
df = pd.DataFrame(items)
df = df[['title', 'link', 'published', 'source']]
df = df.reset_index().rename(columns={'index': 'id'})
df

,id,title,link,published,source
0,0,Oscar Piastri casts clear verdict on controver...,https://news.google.com/rss/articles/CBMiTWh0d...,"Mon, 25 Mar 2024 20:04:26 GMT","{'href': 'https://www.planetf1.com', 'title': ..."
1,1,Oscar Piastri: “We made right call” | Separate...,https://news.google.com/rss/articles/CBMibGh0d...,"Sun, 24 Mar 2024 15:00:00 GMT","{'href': 'https://www.crash.net', 'title': 'Cr..."
2,2,Oscar Piastri's stunning response after Aussie...,https://news.google.com/rss/articles/CBMie2h0d...,"Sun, 24 Mar 2024 09:44:07 GMT","{'href': 'https://au.sports.yahoo.com', 'title..."
3,3,‘Went down like a cold flat white’: Oscar told...,https://news.google.com/rss/articles/CBMiqgFod...,"Sun, 24 Mar 2024 07:50:09 GMT","{'href': 'https://www.foxsports.com.au', 'titl..."
4,4,"Piastri: McLaren F1 team orders ""completely fa...",https://news.google.com/rss/articles/CBMibGh0d...,"Sun, 24 Mar 2024 07:57:35 GMT","{'href': 'https://www.motorsport.com', 'title'..."
...,...,...,...,...,...
94,94,Red Bull pace scrutinised by Oscar Piastri as ...,https://news.google.com/rss/articles/CBMiVWh0d...,"Mon, 04 Mar 2024 08:00:00 GMT","{'href': 'https://www.planetf1.com', 'title': ..."
95,95,F1 news: McLaren acquire Mercedes ‘rear wing e...,https://news.google.com/rss/articles/CBMiYmh0d...,"Sun, 10 Mar 2024 08:00:00 GMT","{'href': 'https://www.planetf1.com', 'title': ..."
96,96,Piastri on his unusual route into racing and j...,https://news.google.com/rss/articles/CBMib2h0d...,"Fri, 12 Jan 2024 08:00:00 GMT","{'href': 'https://www.formula1.com', 'title': ..."
97,97,Oscar Piastri excited for Formula One Australi...,https://news.google.com/rss/articles/CBMiTmh0d...,"Sun, 17 Mar 2024 07:00:00 GMT","{'href': 'https://www.drive.com.au', 'title': ..."


In [37]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [38]:
def polarity_scores_roberta(example):
    encoded_text = tokenizer(example, return_tensors='pt')
    output = model(**encoded_text)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    scores_dict = {
        'negative' : scores[0],
        'neutral' : scores[1],
        'positive' : scores[2]
    }
    return scores_dict

In [42]:
res = {}
for i, row in tqdm(df.iterrows(), total=len(df)):
    try:
        text = row['title']
        myid = row['id']
        roberta_result = polarity_scores_roberta(text)
        res[myid] = roberta_result
    except RuntimeError:
        print(f'Broke for id {myid}')

100%|██████████| 99/99 [00:04<00:00, 21.48it/s]


In [43]:
results_df = pd.DataFrame(res).T
results_df = results_df.reset_index().rename(columns={'index': 'id'})
results_df = results_df.merge(df, how='left')

In [44]:
results_df

,id,negative,neutral,positive,title,link,published,source
0,0,0.067959,0.882304,0.049736,Oscar Piastri casts clear verdict on controver...,https://news.google.com/rss/articles/CBMiTWh0d...,"Mon, 25 Mar 2024 20:04:26 GMT","{'href': 'https://www.planetf1.com', 'title': ..."
1,1,0.147255,0.783936,0.068809,Oscar Piastri: “We made right call” | Separate...,https://news.google.com/rss/articles/CBMibGh0d...,"Sun, 24 Mar 2024 15:00:00 GMT","{'href': 'https://www.crash.net', 'title': 'Cr..."
2,2,0.059492,0.477069,0.463439,Oscar Piastri's stunning response after Aussie...,https://news.google.com/rss/articles/CBMie2h0d...,"Sun, 24 Mar 2024 09:44:07 GMT","{'href': 'https://au.sports.yahoo.com', 'title..."
3,3,0.332271,0.631291,0.036438,‘Went down like a cold flat white’: Oscar told...,https://news.google.com/rss/articles/CBMiqgFod...,"Sun, 24 Mar 2024 07:50:09 GMT","{'href': 'https://www.foxsports.com.au', 'titl..."
4,4,0.016880,0.772762,0.210359,"Piastri: McLaren F1 team orders ""completely fa...",https://news.google.com/rss/articles/CBMibGh0d...,"Sun, 24 Mar 2024 07:57:35 GMT","{'href': 'https://www.motorsport.com', 'title'..."
...,...,...,...,...,...,...,...,...
94,94,0.298376,0.668200,0.033425,Red Bull pace scrutinised by Oscar Piastri as ...,https://news.google.com/rss/articles/CBMiVWh0d...,"Mon, 04 Mar 2024 08:00:00 GMT","{'href': 'https://www.planetf1.com', 'title': ..."
95,95,0.034499,0.916323,0.049178,F1 news: McLaren acquire Mercedes ‘rear wing e...,https://news.google.com/rss/articles/CBMiYmh0d...,"Sun, 10 Mar 2024 08:00:00 GMT","{'href': 'https://www.planetf1.com', 'title': ..."
96,96,0.041081,0.875905,0.083014,Piastri on his unusual route into racing and j...,https://news.google.com/rss/articles/CBMib2h0d...,"Fri, 12 Jan 2024 08:00:00 GMT","{'href': 'https://www.formula1.com', 'title': ..."
97,97,0.001111,0.066903,0.931986,Oscar Piastri excited for Formula One Australi...,https://news.google.com/rss/articles/CBMiTmh0d...,"Sun, 17 Mar 2024 07:00:00 GMT","{'href': 'https://www.drive.com.au', 'title': ..."
